In [2]:
import re
import pyarrow as pa
import pyarrow.parquet as pq

def process_and_save_to_parquet_streaming(input_path: str, output_path: str, batch_size: int = 100_000):
    """
    Processes a huge text file into sentences and writes to Parquet in streaming batches.
    """
    buffer = ""
    sentences_data = []
    sentence_regex = re.compile(r'(?<=[.?!])\s+')

    schema = pa.schema([
        ('id', pa.int64()),
        ('sentence', pa.string())
    ])

    writer = None
    counter = 0

    print("Starting streaming processing...")

    with open(input_path, 'r', encoding='utf-8') as infile:
        for line in infile:
            buffer += line.strip() + " "
            sentences = sentence_regex.split(buffer)
            buffer = sentences.pop()

            for sentence in sentences:
                words = re.findall(r'\S+', sentence)
                if len(words) >= 5:
                    sentences_data.append((counter, sentence.strip()))
                    counter += 1

            # When batch is full, flush to Parquet
            if len(sentences_data) >= batch_size:
                ids, texts = zip(*sentences_data)
                table = pa.Table.from_arrays([pa.array(ids), pa.array(texts)], schema=schema)

                if writer is None:
                    writer = pq.ParquetWriter(output_path, schema, compression="snappy")
                writer.write_table(table)

                sentences_data.clear()  # free memory

        # Handle leftovers
        if buffer.strip():
            words = re.findall(r'\S+', buffer)
            if len(words) >= 5:
                sentences_data.append((counter, buffer.strip()))
                counter += 1

        if sentences_data:
            ids, texts = zip(*sentences_data)
            table = pa.Table.from_arrays([pa.array(ids), pa.array(texts)], schema=schema)
            if writer is None:
                writer = pq.ParquetWriter(output_path, schema, compression="snappy")
            writer.write_table(table)

    if writer:
        writer.close()

    print(f"✅ Done. Total sentences written: {counter}")
    print(f"Parquet file: {output_path}")


In [3]:
process_and_save_to_parquet_streaming("tokenized_output.txt","sentences.parquet")

Starting streaming processing...
✅ Done. Total sentences written: 63706881
Parquet file: sentences.parquet


In [4]:
import pyarrow.parquet as pq
import pandas as pd

def print_first_100_rows(parquet_path: str):
    parquet_file = pq.ParquetFile(parquet_path)

    rows_to_read = 100
    rows_read = 0

    pd.set_option("display.max_colwidth", None)  # show full text, not "..."

    for i in range(parquet_file.num_row_groups):
        row_group = parquet_file.read_row_group(i)
        df = row_group.to_pandas()

        needed = rows_to_read - rows_read
        print(df.head(needed).to_string(index=False))  # print cleanly, no index

        rows_read += min(needed, len(df))
        if rows_read >= rows_to_read:
            break

# Example usage
output_file_path = "sentences.parquet"
print_first_100_rows(output_file_path)


 id                                                                                                                                                                                   sentence
  0                                                                                                            అమెరికా అధ్యక్షుడు డొనాల్డ్ ట్రంప్ కు రాష్ట్రపతి భవన్ వద్ద ఘనస్వాగతం లభించింది.
  1                                                                    ఆయనకు రాష్ట్రపతి రామ్ నాథ్ కోవింద్ దంపతులు ప్రధాని మోదీ సాదరంగా ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు.
  2                                                                                                             ఇటు తెలంగాణలో కరోనా వైరస్ కారణంగా అన్ని దేవాలయాల్లో ముందస్తు చర్యలు చేపట్టారు.
  3                                                                                                                                                 భద్రాద్రి రాముడికి కరోనా ఎఫెక్ట్ తగిలింది.
  4                                          